In [1]:
from time import time
t00 = time()
import numpy as np

In [2]:
X_fn = './data/X_2018clinical.npy'
y_fn = './data/y_2018clinical.npy'
X = np.load(X_fn)
y = np.load(y_fn)
print(X.shape, y.shape)

(10000, 1000) (10000,)


In [3]:
from resnet import ResNet
import os
import torch

In [4]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

In [7]:
# Load trained weights
cnn = ResNet(hidden_sizes, num_blocks, input_dim,
            in_channels=in_channels, n_classes=n_classes)

if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    './best_model.pth', map_location=lambda storage, loc: storage))

<All keys matched successfully>

In [8]:
from training import get_predictions
from datasets import spectral_dataloader

In [9]:
# Make predictions on subset of data
t0 = time()
dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl, cuda)
print('Predicted {} spectra: {:0.2f}s'.format(len(y_hat), time()-t0))

Predicted 10000 spectra: 104.75s


In [10]:
# Computing accuracy
acc = (y_hat == y).mean()
print('Accuracy: {:0.1f}%'.format(100*acc))

Accuracy: 0.9%
